In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=EzBaSeNfpGPABgJc5pzuHzRd9Gs7mzEiv_GtLenZCRE&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5tt_u4aZStMvIz0KYEMfZsI9DYsw5eXIkVmjTFTVgs-cDZVP4d8Ys

Successfully saved authorization token.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
#please adjust this to your scripts path
sys.path.insert(0,'/content/drive/My Drive/s1_ard_py')

In [4]:
roi = ee.Geometry.Polygon(
        [[[105.09332131790751, 11.489785285118867],
          [105.09332131790751, 11.18278399685024],
          [105.40917825150126, 11.18278399685024],
          [105.40917825150126, 11.489785285118867]]])

In [9]:
params = {  'START_DATE': '2018-01-01', 
            'STOP_DATE': '2018-01-15',        
            'ORBIT': 'DESCENDING',
            'POLARIZATION': 'VVVH',
            'ROI': roi,
            'DEM': ee.Image('USGS/SRTMGL1_003'),
            'APPLY_BORDER_NOISE_CORRECTION': False,
            'APPLY_TERRAIN_FLATTENING': True,
            'TERRAIN_FLATTENING_MODEL': 'VOLUME',
            'TERRAIN_FLATTENING_ADDITIONAL_LAYOVER_SHADOW_BUFFER':0,
            'APPLY_SPECKLE_FILTERING': True,
            'SPECKLE_FILTER_FRAMEWORK':'MULTI',
            'SPECKLE_FILTER': 'BOXCAR',
            'SPECKLE_FILTER_KERNEL_SIZE': 7,
            'NR_OF_IMAGES':10,
            'FORMAT': 'DB',
            'CLIP_TO_ROI': False,
            'SAVE_ASSET': False, 
            'ASSET_ID': "users/adugnagirma"
            }

In [10]:
#processed s1 collection
import wrapper as wp
s1_processed = wp.s1_preproc(params)

In [11]:
#Visualize
import folium
import helper
if (params['POLARIZATION']=='VVVH'):
    if (params['FORMAT'] == 'DB'):
      s1_preprocces_view = s1_processed.map(helper.add_ratio_lin).map(helper.lin_to_db2);
      #s1_view = s1.map(helper.add_ratio_lin).map(helper.lin_to_db2);
      visparam = {'bands':['VV', 'VH','VVVH_ratio'],'min': [-20, -25, 1],'max': [0, -5, 15]}
if (params['POLARIZATION']=='VVVH'):
    if (params['FORMAT'] == 'LINEAR'):
      s1_preprocces_view = s1_processed.map(helper.add_ratio_lin)
      #s1_view = s1.map(helper.add_ratio_lin)
      visparam = {'bands':['VV', 'VH','VVVH_ratio'],'min': [0, 0, 0],'max': [0.5, 0.5, 20]}
if (params['POLARIZATION']!='VVVH'):
    if (params['FORMAT'] == 'DB'):
      s1_preprocces_view = s1_processed.map(helper.lin_to_db)
      #s1_view = s1.map(helper.lin_to_db)
      visparam = {'bands':params['POLARIZATION'],'min': -25,'max': 0}
if (params['POLARIZATION']!='VVVH'):
    if (params['FORMAT'] == 'LINEAR'):
      s1_preprocces_view = s1_processed;
      #s1_view = s1;
      visparam = {'bands':params['POLARIZATION'],'min': 0,'max': 0.2}

# Use folium to visualize the imagery.
s1_processed_mapid = s1_preprocces_view.first().getMapId(visparam)
#s1_mapid = s1_view.first().getMapId(visparam)
map = folium.Map(location=[11.48, 105.09])

folium.TileLayer(
    tiles=s1_processed_mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Preprocessed S1',
  ).add_to(map)
map.add_child(folium.LayerControl())
map
